## Important – Do not use in production, for demonstration purposes only – please review the legal notices before continuing
 License agreement: https://github.com/microsoft/Azure-Analytics-and-AI-Engagement/blob/main/HealthCare/License.md 


## Legal Notices
This presentation, demonstration, and demonstration model are for informational purposes only. Microsoft makes no warranties, express or implied, in this presentation demonstration, and demonstration model. Nothing in this presentation, demonstration, or demonstration model modifies any of the terms and conditions of Microsoft’s written and signed agreements. This is not an offer and applicable terms and the information provided is subject to revision and may be changed at any time by Microsoft.

This presentation, demonstration, and/or demonstration model do not give you or your organization any license to any patents, trademarks, copyrights, or other intellectual property covering the subject matter in this presentation, demonstration, and demonstration model.

The information contained in this presentation, demonstration and demonstration model represent the current view of Microsoft on the issues discussed as of the date of presentation and/or demonstration, and the duration of your access to the demonstration model. Because Microsoft must respond to changing market conditions, it should not be interpreted to be a commitment on the part of Microsoft, and Microsoft cannot guarantee the accuracy of any information presented after the date of presentation and/or demonstration and for the duration of your access to the demonstration model.

No Microsoft technology, nor any of its component technologies, including the demonstration model, is intended or made available: (1) as a medical device; (2) for the diagnosis of disease or other conditions, or in the cure, mitigation, treatment or prevention of a disease or other conditions; or (3) as a substitute for the professional clinical advice, opinion, or judgment of a treating healthcare professional. Partners or customers are responsible for ensuring the regulatory compliance of any solution they build using Microsoft technologies.

© 2020 Microsoft Corporation. All rights reserved


## Please don't run / don't click "Run all" the notebook:
At the time of writing of this document, the current core limit is 200 cores per workspace and depending upon number of concurrent users, you may end up with core capacity being exceeded or maximum number of parallel jobs being exceeded error.         



In [31]:
fil="abfss://iomt-data@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/test.csv"

In [32]:
df = spark.read.load(fil, format='csv',header=True)

**Set Up HyperSpace**


In [33]:
from hyperspace import *
hs=Hyperspace(spark)

In [36]:
#Soft-delete the given index for vacuum.
hs.deleteIndex("index")
#Hard-delete the given index and physically remove it from filesystem.
hs.vacuumIndex("index")

In [37]:
from hyperspace import IndexConfig
hs.createIndex(df, IndexConfig("index", ["PatientId"], ["BodyTemperature"]))

In [8]:
display(hs.indexes())

In [38]:
Hyperspace.enable(spark)
df.show()

+--------------------+----------+--------------------+---------------+---------+-------------+----+----------------+-----------------+-------------+------------+--------------------------+--------+---+---------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+
|           patientId|patientAge|            datetime|bodyTemperature|heartRate|breathingRate|spo2|systolicPressure|diastolicPressure|numberOfSteps|activityTime|numberOfTimesPersonStoodUp|calories|vo2|SyntheticPartitionKey|                  id|                _rid|               _self|               _etag|_attachments|       _ts|
+--------------------+----------+--------------------+---------------+---------+-------------+----+----------------+-----------------+-------------+------------+--------------------------+--------+---+---------------------+--------------------+--------------------+--------------------+--------------------+------------+----------+
|005

In [39]:
df = df.filter("PatientId = '0058a52a-235c-11eb-be74-70b5e8b8edbb'").select("PatientId","BodyTemperature")

In [40]:
hs.explain(df, verbose = True)

Plan with indexes:
Project [PatientId#476, BodyTemperature#479]
+- Filter (isnotnull(PatientId#476) && (PatientId#476 = 0058a52a-235c-11eb-be74-70b5e8b8edbb))
   <----+- FileScan parquet [patientId#476,bodyTemperature#479] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://healthcare@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/synapse/workspaces/syn..., PartitionFilters: [], PushedFilters: [IsNotNull(patientId), EqualTo(patientId,0058a52a-235c-11eb-be74-70b5e8b8edbb)], ReadSchema: struct<patientId:string,bodyTemperature:string>---->

Plan without indexes:
Project [PatientId#476, BodyTemperature#479]
+- Filter (isnotnull(PatientId#476) && (PatientId#476 = 0058a52a-235c-11eb-be74-70b5e8b8edbb))
   <----+- FileScan csv [patientId#476,bodyTemperature#479] Batched: false, Format: CSV, Location: InMemoryFileIndex[abfss://iomt-data@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/test.csv], PartitionFilters: [], PushedFilters: [IsNotNull(patientId), EqualTo(patientId,0058a52a-235c

## Data Transformation


In [41]:
%%pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np
pd_df = df.select("PatientId","BodyTemperature").toPandas()
pd_df.groupby(['PatientId'])
print(pd_df)

PatientId BodyTemperature
0  0058a52a-235c-11eb-be74-70b5e8b8edbb            97.5
1  0058a52a-235c-11eb-be74-70b5e8b8edbb            98.7
2  0058a52a-235c-11eb-be74-70b5e8b8edbb            97.4
3  0058a52a-235c-11eb-be74-70b5e8b8edbb            98.6
4  0058a52a-235c-11eb-be74-70b5e8b8edbb            97.1

In [42]:
%%pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np
pd_df = df.select("PatientId","BodyTemperature").toPandas()
pd_df.groupby(['PatientId','BodyTemperature']).sum()

pd_df.head(10)

PatientId BodyTemperature
0  0058a52a-235c-11eb-be74-70b5e8b8edbb            97.5
1  0058a52a-235c-11eb-be74-70b5e8b8edbb            98.7
2  0058a52a-235c-11eb-be74-70b5e8b8edbb            97.4
3  0058a52a-235c-11eb-be74-70b5e8b8edbb            98.6
4  0058a52a-235c-11eb-be74-70b5e8b8edbb            97.1

In [17]:
%%pyspark
df = spark.createDataFrame(pd_df)
df.show(5)

(df
 .coalesce(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .format("com.databricks.spark.csv")
 .save('abfss://iomt-data@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/iomtData'))


+--------------------+----------+---------------+---------+-------------+-------------+--------+
|           PatientId|PatientAge|BodyTemperature|HeartRate|BreathingRate|numberOfSteps|Calories|
+--------------------+----------+---------------+---------+-------------+-------------+--------+
|0058a52a-235c-11e...|        42|           97.5|      157|           47|         2685|    2695|
|0058a52a-235c-11e...|        42|           98.7|      147|          124|         4639|    2922|
|0058a52a-235c-11e...|        42|           97.4|       82|           97|        11649|    2399|
|0058a52a-235c-11e...|        42|           98.6|       59|          159|         4504|    2682|
|0058a52a-235c-11e...|        42|           97.1|      130|           34|         5954|    2636|
+--------------------+----------+---------------+---------+-------------+-------------+--------+
only showing top 5 rows